# Describtion of the model

Data cleaning is represented in the comments.

Firstly, I train a linear_model on the scaled data and adding the predicted result to the original data - give it to XGB !

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [20]:
train_data_pd = pd.read_csv('C:/Users/artur/Downloads/train.csv')
test_data_pd = pd.read_csv('C:/Users/artur/Downloads/test.csv')

In [21]:
train_data = train_data_pd.values
test_data = test_data_pd.values

target = train_data[:, -1]
train_data = train_data[:, :-1]

data = np.row_stack((train_data, test_data))


############################
#   Forming year feature   #
############################
years = np.array([int(year[:4]) for year in data[:, 1]])

data = data[:, 2:-1]
data = np.column_stack((years, data))

data.shape

(38133, 289)

In [22]:
#######################################
#   Deleting NaN-containing cloumns   #
#######################################

print(data.shape)

data[data != data] = "contains nan"
delete_column = []
for j in range(data.shape[1]):
    if "contains nan" in data[:, j]:
        delete_column.append(j)

data = np.delete(data, delete_column, 1)

print(data.shape)

(38133, 289)
(38133, 236)


In [23]:
###################################
#   Using "pd.get_dummies" here   #
###################################

print('data.shape = {}'.format(data.shape), end='\n\n')

will_be_deleted = []
transformed_data = np.zeros((data.shape[0]))
for i, val in enumerate(data[0]):
    if type(val) == str:
        hold = pd.get_dummies(data[:, i]).values
        transformed_data = np.column_stack((transformed_data, hold))
        will_be_deleted.append(i)
        print('index:', i, 'added columns:',  hold.shape[1])

transformed_data = transformed_data[:, 1:]
data = np.delete(data, will_be_deleted, 1)
data = np.column_stack((data, transformed_data))

print('\n')
print('data.shape = {}'.format(data.shape))


data.shape = (38133, 236)

index: 2 added columns: 146
index: 16 added columns: 2
index: 20 added columns: 2
index: 21 added columns: 2
index: 22 added columns: 2
index: 23 added columns: 2
index: 24 added columns: 2
index: 25 added columns: 2
index: 26 added columns: 2
index: 27 added columns: 2
index: 72 added columns: 2
index: 80 added columns: 2
index: 84 added columns: 2
index: 118 added columns: 5


data.shape = (38133, 397)


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

ridge = Ridge()
parameters = {'alpha': [0.1, 2, 3, 5, 12, 30, 100]}

linear_model = GridSearchCV(ridge, parameters)

           ########################
           #   Scaling features   #
##############################################
mean_vec = np.array([np.mean(data[:, j]) for j in range(data.shape[1])])
std_vec = np.array([np.std(data[:, j]) for j in range(data.shape[1])])

data_scaled = (data - mean_vec) / (std_vec+1e-10)
##############################################


train_data = data[:target.shape[0]]
test_data = data[target.shape[0]:]

train_data_scaled = data_scaled[:target.shape[0]]
test_data_scaled = data_scaled[target.shape[0]:]

linear_model.fit(X=train_data_scaled, y=target)

new_column_train = linear_model.predict(X=train_data_scaled)
new_column_test = linear_model.predict(X=test_data_scaled)

train_data = np.column_stack((train_data, new_column_train))
test_data = np.column_stack((test_data, new_column_test))

{'alpha': 100}
(30471, 397)
(30471, 398)


(0, 5850000)
(1, 6000000)
(2, 5700000)
(3, 13100000)
(4, 16331452)
(5, 9100000)
(6, 5500000)
(7, 2000000)
(8, 5300000)
(9, 2000000)

(0, 5046378.756772512)
(1, 5609713.043649767)
(2, 5236538.004253147)
(3, 6499662.985637389)
(4, 10668185.800968759)
(5, 5247386.665156876)
(6, 8092978.745479906)
(7, 6008246.338668449)
(8, 5975691.070100578)
(9, 7690761.600411292)


In [27]:
model = XGBRegressor(max_depth=5)
model.fit(X=train_data, y=target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [28]:
def mean_squared_log_error(real, predicted):
    s=0.0
    for i in range(len(predicted)):
        if predicted[i] < 0 or real[i] < 0:
            continue
        p = np.log(predicted[i]+1)
        r = np.log(real[i]+1)
        s = s + (p - r)**2
    return (s/len(predicted))**0.5

mean_squared_log_error(target, model.predict(train_data))

0.4545441953672685

In [ ]:
test_predicted = model.predict(test_data)

result = pd.read_csv('C:/Users/artur/Downloads/sample_submission.csv')
result['price_doc'] = test_predicted

result.to_csv('submission.csv', index=False)

## I sent the file to kaggle and got 0.35 score
### (Winning score was 0.30)